[[이유한님] 캐글 코리아 캐글 스터디 커널 커리큘럼](https://kaggle-kr.tistory.com/32)  
[타이타닉 캐글 링크](https://www.kaggle.com/c/titanic)

# [타이타닉 튜토리얼 1 - Exploratory data analysis, visualization, machine learning](https://kaggle-kr.tistory.com/17?category=868316)
앞으로 우리가 해야할 프로세스는 대략 아래와 같습니다.

1. 데이터셋 확인 - 대부분의 캐글 데이터들은 잘 정제되어 있습니다. 하지만 가끔 null data가 존재합니다. 이를 확인하고, 향후 수정합니다.
  
2. 탐색적 데이터 분석(exploratory data analysis) - 여러 feature 들을 개별적으로 분석하고, feature 들 간의 상관관계를 확인합니다. 여러 시각화 툴을 사용하여 insight를 얻습니다.
  
3. feature engineering - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature 들을 engineering 합니다. one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 합니다.
4. model 만들기 - sklearn 을 사용해 모델을 만듭니다. 파이썬에서 머신러닝을 할 때는 sklearn 을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있습니다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수 도 있습니다.
  
5. 모델 학습 및 예측 - trainset 을 가지고 모델을 학습시킨 후, testset 을 가지고 prediction 합니다.
  
6. 모델 평가 - 예측 성능이 원하는 수준인지 판단합니다. 풀려는 문제에 따라 모델을 평가하는 방식도 달라집니다. 학습된 모델이 어떤 것을 학습하였는 지 확인해봅니다.

In [1]:
import numpy as np # numpy: arrray(다차원 벡터 자료구조) 및 각종 계산 라이브러리
import pandas as pd # pandas: Series(1차원) 및 DataFrame(2차원) 자료구조 및 각종 데이터 분석 라이브러리
import matplotlib as mpl        # matplotlib: 그래프 등 시각화 라이브러리
import matplotlib.pyplot as plt # matplotlib.pyplot: matplotlib 중 실제 그래프 그리는 데 사용하는 패키지
import seaborn as sns           # seaborn: 시각화 라이브러리 matplotlib에 연계.
# %matplotlib inline
get_ipython().run_line_magic('matplotlib', 'inline')
mpl.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

## plt.style.use('seaborn')
## sns.set(font_scale=2.5) # 이 두줄은 본 필자가 항상 쓰는 방법입니다. matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편합니다.
import missingno as msno 

## ignore warnings
import warnings
warnings.filterwarnings('ignore')


# import numpy as np, pandas as pd, matplotlib as mpl, seaborn as sns, warnings, missingno as msno
# import matplotlib.pyplot as plt

# get_ipython().run_line_magic('matplotlib', 'inline')
# mpl.rc('font', family='Malgun Gothic')
# plt.rcParams['axes.unicode_minus'] = False
# warnings.filterwarnings('ignore')

ImportError: DLL load failed while importing _multiarray_umath: 지정된 모듈을 찾을 수 없습니다.

ImportError: numpy._core.multiarray failed to import

## 1. Dataset 확인
- 파이썬에서 테이블화 된 데이터를 다루는 데 가장 최적화되어 있으며, 많이 쓰이는 라이브러리는 pandas 입니다.
- 우리는 pandas 를 사용하여 데이터셋의 간단한 통계적 분석 부터, 복잡한 처리들을 간단한 메소드를 사용하여 해낼 수 있습니다.
- 파이썬으로 데이터분석을 한다고 하면 반드시 능숙해져야 할 라이브러리이니, 여러 커널들을 공부하시면서 사용법에 익숙해지도록 반복 또 반복하시길 권장합니다.
- 캐글에서 데이터셋은 보통 train, testset 으로 나뉘어 있습니다.

In [ ]:
df_train = pd.read_csv('./input/001_titanic/train.csv')
df_test = pd.read_csv('./input/001_titanic/test.csv')

In [ ]:
df_train.head()

In [ ]:
## pandas dataframe 에는 describe() 메소드가 있는 데, 
## 이를 쓰면 각 feature 가 가진 통계치들을 반환해줍니다.
df_train.describe()

### 1.1 Null data check

In [ ]:
for col in df_train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

print('='*55)

for col in df_test.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

# for col in df_train.columns:
#     msg = f'column: {col:>10}\t Percent of NaN value: {100*(df_train[col].isnull().sum()/df_train[col].shape[0]):.2f}%'
#     print(msg)

# print ('='*55)

# for col in df_test.columns:
#     msg = f'column: {col:>10}\t Percent of NaN value: {100*(df_test[col].isnull().sum()/df_test[col].shape[0]):.2f}%'
#     print(msg)

In [ ]:
# 나는 보통 info()로 확인
print(df_train.info(), df_test.info());

In [ ]:
msno.matrix(df=df_train.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2));

# 그래프는 출력되지만 아래와 같은 오류 발생.
# ValueError: keyword grid_b is not recognized; valid keywords are ['size', 'width', 'color', 'tickdir', 'pad', 'labelsize', 'labelcolor', 'zorder', 'gridOn', 'tick1On', 'tick2On', 'label1On', 'label2On', 'length', 'direction', 'left', 'bottom', 'right', 'top', 'labelleft', 'labelbottom', 'labelright', 'labeltop', 'labelrotation', 'grid_agg_filter', 'grid_alpha', 'grid_animated', 'grid_antialiased', 'grid_clip_box', 'grid_clip_on', 'grid_clip_path', 'grid_color', 'grid_dash_capstyle', 'grid_dash_joinstyle', 'grid_dashes', 'grid_data', 'grid_drawstyle', 'grid_figure', 'grid_fillstyle', 'grid_gapcolor', 'grid_gid', 'grid_in_layout', 'grid_label', 'grid_linestyle', 'grid_linewidth', 'grid_marker', 'grid_markeredgecolor', 'grid_markeredgewidth', 'grid_markerfacecolor', 'grid_markerfacecoloralt', 'grid_markersize', 'grid_markevery', 'grid_mouseover', 'grid_path_effects', 'grid_picker', 'grid_pickradius', 'grid_rasterized', 'grid_sketch_params', 'grid_snap', 'grid_solid_capstyle', 'grid_solid_joinstyle', 'grid_transform', 'grid_url', 'grid_visible', 'grid_xdata', 'grid_ydata', 'grid_zorder', 'grid_aa', 'grid_c', 'grid_ds', 'grid_ls', 'grid_lw', 'grid_mec', 'grid_mew', 'grid_mfc', 'grid_mfcalt', 'grid_ms']
# 해결법: ~/env/Lib/site-packages/missingno/missingno.py에서 .grid(b=False)를 .grid(visible=False)로 변경 후 저장.
# matplotlib.axes._base.py에서 인자명이 변경됨.
# 출처: 스택오버플로우(https://stackoverflow.com/questions/75525029/msno-matrix-shows-an-error-when-i-use-any-venv-using-pyenv)

In [ ]:
msno.bar(df=df_train.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2));

In [ ]:
msno.bar(df=df_test.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2));

### 1.2 Target label 확인

- target label 이 어떤 distribution 을 가지고 있는 지 확인해봐야 합니다.

- 지금 같은 binary classification 문제의 경우에서, 1과 0의 분포가 어떠냐에 따라 모델의 평가 방법이 달라 질 수 있습니다.

In [ ]:
df_train.head()

In [ ]:
df_train.Survived

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 8)) # 한 그림에 여러 그래프: 1행 2열, 전체 크기 (18, 8)

df_train['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%.1f%%', ax=ax[0], shadow=True)    # df에서 바로 plot 그리기
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot(data=df_train, x='Survived', ax=ax[1])    # seaborn으로 plot 그리기
ax[1].set_title('Count plot - Survived')
plt.show()

## 2. Exploratory data analysis

- 이제 본격적으로 데이터 분석을 해보겠습니다. 데이터는 매우 많습니다. 이 많은 데이터 안에 숨겨진 사실을 찾기 위해선 적절한 시각화가 필요합니다.  
  
- 시각화 라이브러리는 matplotlib, seaborn, plotly 등이 있습니다. 특정 목적에 맞는 소스코드를 정리해두어 필요할 때마다 참고하면 편합니다.

### 2.1 Pclass

- 먼저 Pclass 에 대해서 살펴보겠습니다. Pclass 는 ordinal, 서수형 데이터입니다. 카테고리이면서, 순서가 있는 데이터 타입입니다.  
    
- 먼저 Pclass 에 따른 생존률의 차이를 살펴보겠습니다. 엑셀의 피벗 차트와 유사한 작업을 하게 되는데, pandas dataframe 에서는 groupby 를 사용하면 쉽게 할 수 있습니다. 또한 pivot 이라는 메소드도 있습니다.  
  
- 'Pclass', 'Survived' 를 가져온 후, pclass 로 묶습니다. 그러고 나면 각 pclass 마다 0, 1 이 count가 되는데, 이를 평균내면 각 pclass 별 생존률이 나옵니다  
  
- 아래와 같이 count() 를 하면, 각 class 에 몇명이 있는 지 확인할 수 있으며, sum() 을 하면, 216 명중 생존한(survived=1)사람의 총합을 주게 됩니다

In [ ]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).agg(['count', 'sum', 'mean'])
# as_index: 집계 기준 column을 index 사용 여부. default=True
df_train[['Pclass', 'Survived']].groupby('Pclass').agg(['count','sum','mean'])

In [ ]:
df_train[['Pclass', 'Survived']].pivot_table(index='Pclass', values='Survived', aggfunc=['count', 'sum', 'mean'])

In [ ]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).sum()

In [ ]:
df_train[['Pclass', 'Survived']].pivot_table(index='Pclass', values='Survived', aggfunc='sum')

In [ ]:
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')
# margins: 소계 및 합계

In [ ]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar();
# df_train[['Pclass', 'Survived']].pivot_table(index='Pclass', values='Survived', aggfunc='mean').sort_values(by='Survived', ascending=False).plot.bar();

### 2.2 Sex
이번에는 성별로 생존률이 어떻게 달라지는 지 확인해보겠습니다.  
마찬가지로 pandas groupby 와 seaborn countplot 을 사용해서 시각화해봅시다.  

In [ ]:
y_position = 1.02
fig, ax = plt.subplots(1, 2, figsize=(18, 8))
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0]) # value_counts를 이용하여 bar로 countplot 생성
ax[0].set_title('Number of Passengers By Pclass', y=y_position) # y: 제목과 그래프의 거리
ax[0].set_ylabel('Count')
sns.countplot(data=df_train, x='Pclass', hue='Survived', ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y=y_position)
plt.show()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot(data=df_train, x='Sex', hue='Survived', ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

# fig, ax = plt.subplots(1, 2, figsize=(18,8))
# df_train[['Sex', 'Survived']].pivot_table(index='Sex', values='Survived', aggfunc='mean').plot.bar(ax=ax[0])
# ax[0].set_title('Survived vs Sex')
# sns.countplot(data=df_train, x='Sex', hue='Survived', ax=ax[1])
# ax[1].set_title('Sex: Survived vs Dead')
# plt.show();

In [ ]:
pd.crosstab(df_train['Sex'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

### 2.3 Both Sex and Pclass
- 이번에는 Sex, Pclass 두가지에 관하여 생존이 어떻게 달라지는 지 확인해 봅시다  
  
- seaborn 의 factorplot 을 이용하면, 손쉽게 3개의 차원으로 이루어진 그래프를 그릴 수 있습니다.  

In [ ]:
sns.catplot(data=df_train, x='Pclass', y='Survived', hue='Sex', kind='point',
               aspect=1.5, errwidth=2, scale=1);
            # kind의 인자: 'strip', 'swarm', 'box', 'violin', 'boxen', 'point', 'bar', 'count'

In [ ]:
sns.catplot(data=df_train, x='Sex', y='Survived',col='Pclass', kind='point',
               aspect=1, errwidth=2, scale=1);

### 2.4 Age

이번에는 Age feature 를 살펴봅시다.

In [ ]:
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train['Age'].max()))
print('제일 어린 탑승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))

# print(f'제일 나이 많은 탑승객: {df_train.Age.max():.1f} Years')
# print(f'제일 어린 탑승객: {df_train.Age.min():.1f} Years')
# print(f'탑승객 평균 나이: {df_train.Age.mean():.1f} Years')

In [ ]:
[i*0.001 for i in range(0,31,5)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax=ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax=ax)  # 동일한 ax에 그래프 그리기
plt.legend(['Survived == 1', 'Survived == 0'])
plt.show()

In [ ]:
## Age distribution withing classes
plt.figure(figsize=(8, 6))
df_train['Age'][df_train['Pclass'] == 1].plot(kind='kde')
df_train['Age'][df_train['Pclass'] == 2].plot(kind='kde')
df_train['Age'][df_train['Pclass'] == 3].plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
plt.legend(['1st Class', '2nd Class', '3rd Class'])
plt.show();

Class 가 높을 수록 나이 많은 사람의 비중이 커짐  
나이대가 변하면서 생존률이 어떻게 되는 지 보려고 합니다.  
나이범위를 점점 넓혀가며, 생존률이 어떻게 되는지 한번 봅시다.  

In [ ]:
cummulate_survival_ratio = []
for i in range(1, 80):
    cummulate_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum() / len(df_train[df_train['Age'] < i]['Survived']))
    
plt.figure(figsize=(7, 7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0~x)')
plt.show()

- 보시다시피, 나이가 어릴 수록 생존률이 확실히 높은것을 확인할 수 있습니다.  
 
- 우리는 이 나이가 중요한 feature 로 쓰일 수 있음을 확인했습니다.

### 2.5 Pclass, Sex, Age
  
- 지금까지 본, Sex, Pclass, Age, Survived 모두에 대해서 보고싶습니다. 이를 쉽게 그려주는 것이 seaborn 의 violinplot 입니다.  
  
- x 축은 우리가 나눠서 보고싶어하는 case(여기선 Pclass, Sex) 를 나타내고, y 축은 보고 싶어하는 distribution(Age) 입니다.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 8))
sns.violinplot(data=df_train, x='Pclass', y='Age', hue='Survived', scale='count', split=True, ax=ax[0])
# split: hue가 주어질 경우 True=그래프를 직선 기준으로 좌우로 그림, False=그래프를 별개로 그림
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))

sns.violinplot(data=df_train, x='Sex', y='Age', hue='Survived', scale='count', split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

### 2.6 Embarked

- Embarked 는 탑승한 항구를 나타냅니다.  
  
- 위에서 해왔던 것과 비슷하게 탑승한 곳에 따르 생존률을 보겠습니다.

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(7, 7))
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax);

# df_train[['Embarked', 'Survived']].groupby(['Embarked']).mean().sort_values(by='Survived', ascending=False).plot.bar(figsize=(7,7));
# df_train[['Embarked', 'Survived']].pivot_table(index='Embarked', values='Survived', aggfunc='mean').sort_values(by='Survived', ascending=False).plot.bar(figsize=(7,7));

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20,15))
sns.countplot(data=df_train, x='Embarked', ax=ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Boarded')

sns.countplot(data=df_train, x='Embarked', hue='Sex', ax=ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')

sns.countplot(data=df_train, x='Embarked', hue='Survived', ax=ax[1,0])
ax[1,0].set_title('(3) Embared vs Survived')

sns.countplot(data=df_train, x='Embarked', hue='Pclass', ax=ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

## 2.7 Family - SibSp(형제 자매) + Parch(부모, 자녀)

SibSp와 Parch를 합하면 Family 가 될 것입니다. Family 로 합쳐서 분석해봅시다

In [ ]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 ## 자신을 포함해야 하니 1을 더합니다.
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1 ## 자신을 포함해야 하니 1을 더합니다.
print('Maximun size of Family:', df_train.FamilySize.max())
print('Minimun size of Family:', df_train.FamilySize.min())

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(40,10))
sns.countplot(data=df_train, x='FamilySize', ax=ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y=1.02)

sns.countplot(data=df_train, x='FamilySize', hue='Survived', ax=ax[1])
ax[1].set_title('(2) Survived countplot depending on FamilySize', y=1.02)

df_train[['FamilySize', 'Survived']].groupby(['FamilySize']).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax[2])
# df_train[['FamilySize', 'Survived']].pivot_table(index='FamilySize', values='Survived', aggfunc='mean').sort_values(by='Survived', ascending=False).plot.bar(ax=ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize', y=1.02)

plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show();

### 2.8 Fare  

Fare 는 탑승요금이며, contious feature 입니다. 한번 histogram 을 그려보겠습니다.

In [ ]:
print(df_train.Fare.skew())
df_train.Fare

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df_train['Fare'], color='b', label=f'Skewness: {df_train.Fare.skew(): .2f}', ax=ax)
# df.skew(): 왜도(or 비대칭도), 평균에 대해 최빈값이 치우쳐진 정도를 나타내는 척도
g = g.legend(loc='best')

- 보시다시피, distribution이 매우 비대칭인 것을 알 수 있습니다.(high skewness). 만약 이대로 모델에 넣어준다면 자칫 모델이 잘못 학습할 수도 있습니다. 몇개 없는 outlier 에 대해서 너무 민감하게 반응한다면, 실제 예측 시에 좋지 못한 결과를 부를 수 있습니다.  
  
- outlier의 영향을 줄이기 위해 Fare 에 log 를 취하겠습니다.  
  
- 여기서 우리는 pandas 의 유용한 기능을 사용할 겁니다. dataFrame 의 특정 columns 에 공통된 작업(함수)를 적용하고 싶으면 아래의 map, 또는 apply 를 사용하면 매우 손쉽게 적용할 수 있습니다.  
  
- 우리가 지금 원하는 것은 Fare columns 의 데이터 모두를 log 값 취하는 것인데, 파이썬의 간단한 lambda 함수를 이용해 간단한 로그를 적용하는 함수를 map 에 인수로 넣어주면, Fare columns 데이터에 그대로 적용이 됩니다. 매우 유용한 기능이니 꼭 숙지하세요!

In [ ]:
df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean() ## testset에 있는 NaN value를 평균으로 대치
df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i>0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i>0 else 0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df_train['Fare'], color='b', label=f'Skewness: {df_train.Fare.skew()}', ax=ax)
g.legend(loc='best');

- log 를 취하니, 이제 비대칭성이 많이 사라진 것을 볼 수 있습니다.  
  
- 우리는 이런 작업을 사용해 모델이 좀 더 좋은 성능을 내도록 할 수 있습니다.  
  
- 사실 방금한 것은 feature engineering 에 들어가는 부분인데, 여기서 작업했습니다.  
  
- 모델을 학습시키기 위해, 그리고 그 모델의 성능을 높이기 위해 feature 들에 여러 조작을 가하거나, 새로운 feature를 추가하는 것을 feature engineering 이라고 하는데, 우리는 이제 그것을 살펴볼 것입니다.

### 2.9 Cabin
- 이 feature 는 NaN 이 대략 80% 이므로, 생존에 영향을 미칠 중요한 정보를 얻어내기가 쉽지는 않습니다.  
  
- 그러므로 우리가 세우려는 모델에 포함시키지 않도록 하겠습니다.

### 2.10 Ticket
  
- 이 feature 는 NaN 은 없습니다. 일단 string data 이므로 우리가 어떤 작업들을 해주어야 실제 모델에 사용할 수 있는데, 이를 위해선 사실 아이디어가 필요합니다.

In [ ]:
df_train['Ticket'].value_counts()

- 보시다시피, ticket number 는 매우 다양합니다. 우리는 여기서 어떤 특징을 이끌어내서 생존과 연결시킬 수 있을까요?

- 여러분이 직접 한번 아이디어를 내보세요! 이것이 본격적인 캐글 레이스의 시작점입니다 ^^

- 이 튜토리얼에서는 튜토리얼이니 일단 ticket 은 넘기도록 하겠습니다. 튜토리얼을 끝낸 후, 여러분의 모델의 성능을 향상시키기 위해 ticket 에서 정보를 이끌어내는 것도 좋겠네요!

- 본 튜토리얼은 https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python, https://www.kaggle.com/startupsci/titanic-data-science-solutions, https://www.kaggle.com/ash316/eda-to-prediction-dietanic, https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling 을 참고하여 만들었습니다. 공유해준 캐글러께 감사드립니다.